In [3]:
import os
import json
import pandas as pd
import numpy as np
import json
import logging ### to monitor the code
from bs4 import BeautifulSoup
import xml.etree.cElementTree as ET
import pickle
import math
import random
import sys
import csv
import unicodedata
import requests
from urllib.request import urlopen 
import io
import getpass
import re
from collections import defaultdict
from itertools import islice # to iterate through dicts

import nltk
from nltk.collocations import *

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

import sddk

In [4]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [16]:
# to access gsheet, you need Google Service Account key json file
# I have mine located in my personal space on sciencedata.dk, so I read it from there:

# (1) read the file and parse its content
file_data = conf[0].get("https://sciencedata.dk/files/ServiceAccountsKey.json").json()
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)

ECCE_AGT_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1KPpPaeX215HR_fVrakvJp8aB6oZDhHFTcBw0MKLw6as/edit?usp=sharing")

# Import the data

In [6]:
AGT = sddk.read_file("SDAM_data/AGT/AGT_20201020.json", "df", conf)
AGT.head(5)

,filename,author,title,string,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,wordcount
0,tlg0001.tlg001.perseus-grc2.xml,Apollonius Rhodius,Argonautica; Argonautica,"ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μ...",tlg0001,tlg0001.tlg001,3 B.C.,-2.5,{'-2.5': 1},-2.5,pagan,38822
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War; Historiae in two volumes,Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν ...,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,150160
2,tlg0004.tlg001.perseus-grc1.xml,Diogenes Laertius,Lives of Eminent Philosophers; Lives of Eminen...,ΒΙΩΝ ΚΑΙ ΓΝΩΜΩΝ ΤΩΝ ΕΝ ΦΙΛΟΣΟΦΙΑΙ ΕΥΔΟΚΙΜΗΣΑΝΤ...,tlg0004,tlg0004.tlg001,A.D. 3,2.5,{'2.5': 1},NaN,,111053
3,tlg0005.tlg001.perseus-grc1.xml,Theocritus,Idylls ; Idylls,Θύρσις ἢ ᾠδή Θύρσις ̔Αδύ τι τὸ ψιθύρισμα καὶ ἁ...,tlg0005,tlg0005.tlg001,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,19514
4,tlg0005.tlg002.perseus-grc1.xml,Theocritus,Epigrams ; Idylls,α παλ ϝι.336 τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυ...,tlg0005,tlg0005.tlg002,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,1807


In [7]:
AGTc = AGT[AGT["provenience"]=="christian"]
AGTc.head()

,filename,author,title,string,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,wordcount
379,tlg0031.tlg001.perseus-grc2.xml,None,New Testament - Matthew; Machine readable text...,ΚΑΤΑ ΜΑΘΘΑΙΟΝ ΒΙΒΛΟΣ γενέσεως Ἰησοῦ Χριστοῦ υἱ...,tlg0031a,tlg0031.tlg001,A.D. 1,0.5,{'0.5': 1},0.5,christian,18289
380,tlg0031.tlg002.perseus-grc2.xml,None,New Testament - Mark; Machine readable text; T...,ΚΑΤΑ ΜΑΡΚΟΝ ΑΡΧΗ τοῦ εὐαγγελίου Ἰησοῦ Χριστοῦ ...,tlg0031b,tlg0031.tlg002,A.D. 1,0.5,{'0.5': 1},0.5,christian,11277
381,tlg0031.tlg003.perseus-grc2.xml,Luke (the evangelist),New Testament - Luke; Machine readable text; T...,ΚΑΤΑ ΛΟΥΚΑΝ ΕΠΕΙΔΗΠΕΡ ΠΟΛΛΟΙ ἐπεχείρησαν ἀνατά...,tlg0031luke,tlg0031.tlg003,A.D. 1,0.5,{'0.5': 1},0.5,christian,19460
382,tlg0031.tlg004.perseus-grc2.xml,Johnannine literature,New Testament - John; Machine readable text; T...,"ΚΑΤΑ ΙΩΑΝΗΝ ΕΝ ΑΡΧΗ ἦν ὁ λόγος, καὶ ὁ λόγος ἦν...",tlg0031john,tlg0031.tlg004,A.D. 1,0.5,{'0.5': 1},0.5,christian,15592
383,tlg0031.tlg005.perseus-grc2.xml,Luke (the evangelist),New Testament - Acts; Machine readable text; T...,πράξεις ἀποστόλων τὸν μὲν πρῶτον λόγον ἐποιησά...,tlg0031luke,tlg0031.tlg005,A.D. 1,0.5,{'0.5': 1},0.5,christian,18408


In [8]:
len(AGTc)

147

In [13]:
AGTc.sort_values("date_avr", inplace=True)

/home/kasev/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
set_with_dataframe(ECCE_AGT_overview.add_worksheet("AGTc_works", cols=1, rows=1), AGTc.drop("string", axis=1)) 

In [14]:
len(AGTc["author_id"].unique().tolist())

48

In [15]:
AGTc["author"].unique().tolist()

[None,
 'Clemens Romanus',
 'Johnannine literature',
 'Paul of Tarsus',
 'Luke (the evangelist)',
 'Polycarp',
 'Ignatius Antiochenus',
 'Barnabas',
 'Athenagoras',
 'Justin Martyr',
 'Justinus Martyr',
 'Maximus of Tyre',
 'Anonymus',
 'Dionysius Corinthius',
 'Hermas, 2nd cent.',
 'Tatianus',
 'Theophilus Antiochenus',
 'Irenaeus',
 'Origen',
 'Origenes',
 'Clement of Alexandria',
 'Hippolytus',
 'Anonymous',
 'Gregorius Nazianzenus',
 'Basil, Saint, Bishop of Caesarea',
 'Pseudo-Justinus Martyr',
 'Eusebius',
 'Eusebius of Caesarea',
 'Epiphanius',
 'pseudo-Menander',
 'Athanasius of Alexandria',
 'Athanasius',
 'Theodoretus',
 'Philostorgius',
 'Theodoret, Bishop of Cyrus',
 'Socrates, Scholasticus',
 'Philip Edward Pusey',
 'Sozomenus, Salaminius Hermias',
 'Zosimus',
 'Evagrius, Scholasticus',
 'Joannes Philoponus']